In [13]:
import numpy as np

In [14]:
k_b = 1.38e-16 # Kelvin
a = 7.56e-15 # erg cm^-3
ion = 1.26
electron = 1.15
K = 1e13 # erg cm^-3
m_H = 1.66053907e-24

In [15]:
N_rho = int(input('Enter desired number of points for density: '))
N_T = N_rho #Note: N_rho must be equal to N_T, as such the desired number of points for density will automatically equal the desired number of points for temperature
rho = np.logspace(np.log10(1), np.log10(1e4), num = N_rho)
T = np.logspace(np.log10(1e6), np.log10(1e8), num = N_T)

In [16]:
def pressure(rho, T):
    return ((k_b * rho * T)/(ion * m_H)) + (1/3) * a * T**4 + K*(rho)**(5/3) * electron**(-5/3)

In [17]:
print(f'This is density: {rho}')
print(f'This is temperature: {T}')
print(f'This is pressure: {pressure(rho,T)}')

This is density: [1.00000000e+00 1.42510267e+00 2.03091762e+00 2.89426612e+00
 4.12462638e+00 5.87801607e+00 8.37677640e+00 1.19377664e+01
 1.70125428e+01 2.42446202e+01 3.45510729e+01 4.92388263e+01
 7.01703829e+01 1.00000000e+02 1.42510267e+02 2.03091762e+02
 2.89426612e+02 4.12462638e+02 5.87801607e+02 8.37677640e+02
 1.19377664e+03 1.70125428e+03 2.42446202e+03 3.45510729e+03
 4.92388263e+03 7.01703829e+03 1.00000000e+04]
This is temperature: [1.00000000e+06 1.19377664e+06 1.42510267e+06 1.70125428e+06
 2.03091762e+06 2.42446202e+06 2.89426612e+06 3.45510729e+06
 4.12462638e+06 4.92388263e+06 5.87801607e+06 7.01703829e+06
 8.37677640e+06 1.00000000e+07 1.19377664e+07 1.42510267e+07
 1.70125428e+07 2.03091762e+07 2.42446202e+07 2.89426612e+07
 3.45510729e+07 4.12462638e+07 4.92388263e+07 5.87801607e+07
 7.01703829e+07 8.37677640e+07 1.00000000e+08]
This is pressure: [7.38813378e+13 1.26511446e+14 2.16709109e+14 3.71350419e+14
 6.36586502e+14 1.09170489e+15 1.87298882e+15 3.21480913e

In [18]:
rho_0 = float(input('Enter desired density value in g cm^-3: ')) #Value I used was 7894 
T_0 = float(input('Enter desired temperature value in K: ')) #Value I used was 28900000

rho_i = rho[np.searchsorted(rho,rho_0) - 1]
rho_ip1 = rho[np.searchsorted(rho,rho_0)]
T_j = T[np.searchsorted(T,T_0) - 1]
T_jp1 = T[np.searchsorted(T,T_0)]
print(f'({rho_0}, {T_0})')
print(f'({rho_i}, {T_j})')
print(f'({rho_ip1}, {T_j})')
print(f'({rho_i}, {T_jp1})')
print(f'({rho_ip1}, {T_jp1})')

(7894.0, 28900000.0)
(7017.03828670383, 24244620.170823306)
(10000.0, 24244620.170823306)
(7017.03828670383, 28942661.247167516)
(10000.0, 28942661.247167516)


You can rewrite the solution to a bilinear interpolation problem as a multilinear polynomial:
$$P(\rho, T) \approx a \rho T + b T + c \rho + d$$
where the coefficients are found by solving the linear system:
$$\begin{bmatrix}
1 & \rho_1 & T_1 & \rho_1 T_1 \\
1 & \rho_1 & T_2 & \rho_1 T_2 \\
1 & \rho_2 & T_1 & \rho_2 T_1 \\
1 & \rho_2 & T_2 & \rho_2 T_2  
\end{bmatrix}
\begin{bmatrix} 
d \\
c \\ 
b \\
a
\end{bmatrix} =
\begin{bmatrix}
P(\rho_1, T_1) \\
P(\rho_1, T_2) \\
P(\rho_2, T_1) \\
P(\rho_2, T_2) \\
\end{bmatrix}
$$

In [19]:
L = np.array([
    [1, rho_i, T_j, rho_i*T_j],
    [1, rho_i, T_jp1, rho_i*T_jp1],
    [1, rho_ip1, T_j, rho_ip1*T_j],
    [1, rho_ip1, T_jp1, rho_ip1*T_jp1],
])
m = np.array([pressure(rho_i, T_j), pressure(rho_i, T_jp1), pressure(rho_ip1, T_j), pressure(rho_ip1, T_jp1)])

x = np.linalg.solve(L,m)
a1 = x[3]
b = x[2]
c = x[1]
d = x[0]
print(f'a = {a1}, b = {b}, c = {c}, d= {d}')

a = 65956779.640125364, b = 191059762.0255075, c = 5496565561714537.0, d= -1.8198573010017083e+19


In [20]:
def bilinear_interpolation(a1,b,c,d, rho_0, T_0):
    return a1*(rho_0)*(T_0) + b*(T_0) + c*(rho_0) + d

print(bilinear_interpolation(a1,b,c,d,rho_0,T_0))
print(f'P(rho_i, T_j) < P(rho_0, T_0) < P(rho_ip1, T_jp1): ')
print(f'{pressure(rho_i,T_j)} < {bilinear_interpolation(a1,b,c,d,rho_0,T_0)} < {pressure(rho_ip1,T_jp1)}')

#Relative error from exact answer
print(f'Exact Answer: {pressure(rho_0,T_0)}')
print(f'Relative Error: {((bilinear_interpolation(a1,b,c,d,rho_0,T_0) - pressure(rho_0,T_0)) / pressure(rho_0,T_0)) * 100}%')

4.024399261532743e+19
P(rho_i, T_j) < P(rho_0, T_0) < P(rho_ip1, T_jp1): 
3.159659551807755e+19 < 4.024399261532743e+19 < 5.586225968588079e+19
Exact Answer: 3.984208779350696e+19
Relative Error: 1.0087443808252727%
